# Tutorial of  puzzle solver

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JulienNonin/jigsaw-puzzle-solver/blob/master/sandbox.ipynb)

In [ ]:
! git clone https://github.com/JulienNonin/jigsaw-puzzle-solver.git

In [ ]:
cd jigsaw-puzzle-solver

## Introduction
The idea of our project is to implement a puzzle solver which can generate puzzles in a custom way (according to the number of pieces, size, etc) and solve puzzles in different ways (according to different color spaces, different compatibility calculation methods, etc). 
<br>
Our project is inspired by [Pomeranz, D., Shemesh, M., & Ben-Shahar, O. (2011, June). A fully automated greedy square jigsaw puzzle solver. In CVPR 2011 (pp. 9-16). IEEE](https://www.cs.bgu.ac.il/~ben-shahar/Publications/2011-Pomeranz_Shemesh_and_Ben_Shahar-A_Fully_Automated_Greedy_Square_Jigsaw_Puzzle_Solver.pdf)

## Demonstrate
The following will demonstrate the main function of puzzle solver:

At first, we import all packages we need.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from jigsolver.puzzle import *
from jigsolver.metrics import *
from jigsolver.pomeranz_solver.solver import *
from copy import copy, deepcopy

### Modelisation of a Piece

Here we created a piece with 3x3 pixels, we can get Top border of the piece and Bottom border of the piece easily. The main idea of the puzzle solver is based on comparing the compatibility between pieces.

In [ ]:
np.random.seed(1)
P=Piece(np.random.randint(255,size=(3,3,3)))
Q=Piece(np.random.randint(255,size=(3,3,3)))

plt.figure(figsize=(12,10))

plt.subplot(1,3,1)
plt.imshow(P.picture,alpha=0.5)
plt.title('Example of a Piece')

plt.subplot(1,3,2)
plt.imshow([P.get_border(Border.TOP)],alpha=0.5)
plt.title('Top border of the piece')

plt.subplot(1,3,3)
plt.imshow([P.get_border(Border.BOTTOM)],alpha=0.5)
plt.title('Bottom border of the piece')

<hr/>

### Modelisation of a puzzle

Here we show how to create a Puzzle. We can create it by many ways : 
- Giving a number of pieces specifying the initializer <code>nb_pieces</code>
- Giving a number of rows and number of columns specifying the initializer <code>nb_rows_and_columns</code>
- Giving a particular patch_size specifying the initializer <code>patch</code>

In [ ]:
#import of the image
img_folder = 'img/'
img = plt.imread(img_folder + "eiffel.jpg")

In [ ]:
N=24
rows_and_columns=(9,13)
patch_size=100

plt.figure(figsize=(20,20))

plt.subplot(1,3,1)
eiffel_puzzle = Puzzle(img,nb_pieces(N))
plt.title(f'A puzzle with {N} pieces ')
eiffel_puzzle.ground_truth.display()


plt.subplot(1,3,2)
eiffel_puzzle = Puzzle(img,nb_rows_and_columns(rows_and_columns))
plt.title(f'A puzzle with {rows_and_columns[0]} rows and {rows_and_columns[1]} columns ')
eiffel_puzzle.ground_truth.display()

plt.subplot(1,3,3)
eiffel_puzzle = Puzzle(img,patch(patch_size))
plt.title(f'A puzzle based on patches of lengths {100} ')
eiffel_puzzle.ground_truth.display()

You can easily change from one configuration to an other using the method <code> change_structure </code>

In [ ]:
eiffel_puzzle = Puzzle(img,nb_pieces(N))

plt.figure(figsize=(20,20))

plt.subplot(1,3,1)

eiffel_puzzle.change_structure(patch(200))
plt.title(f'A puzzle based on patches of lengths 200 ')
eiffel_puzzle.ground_truth.display()


plt.subplot(1,3,2)
eiffel_puzzle.change_structure(nb_rows_and_columns((10,10)))
plt.title(f'A puzzle with 10 rows and 10 columns ')
eiffel_puzzle.ground_truth.display()

### Prepare to play

When you create a puzzle, some pieces are created and gathered into a list called bag_of_pieces that you can obtain easily doing <code>MyPuzzle.bag_of_pieces</code>
Immediately after, these pieces are shuffled and the game can start

When the game start, the player ground is empty. We will call this player ground a **Board**

In [ ]:
eiffel_puzzle.display()

Nevertheless, you can retrieve the original arrangement using <code>eiffel_puzzle.ground_truth</code>. 

This is done in order to be able to compute a performance metric which could enable us to compare many solvers based on their performances for some puzzles.

In [ ]:
eiffel_puzzle.ground_truth.display()

When the game has started, the player has to place correctly the pieces from the bag of pieces into the Board.

Here we used an approach using some compatibilities metrics enabling the computer to evaluate how well two pieces matches.

## Placer

Now, let's see how the puzzle solver place the pieces into board. We just create the puzzle and shuffle it as before.

In [ ]:
img = 255 * plt.imread("img/peppers.png")[:,:,:3]
#img = plt.imread("img/eiffel.jpg")
puzzle = Puzzle(patch_size=28)
puzzle.create_from_img(img)
puzzle.display()
puzzle.shuffle()

We compute the compatibility matrix in the paper of [Pomeranz, D., Shemesh, M., & Ben-Shahar, O. (2011, June). A fully automated greedy square jigsaw puzzle solver. In CVPR 2011 (pp. 9-16). IEEE](https://www.cs.bgu.ac.il/~ben-shahar/Publications/2011-Pomeranz_Shemesh_and_Ben_Shahar-A_Fully_Automated_Greedy_Square_Jigsaw_Puzzle_Solver.pdf) <br><br>
The compatibility matrix has the compatibility value of each piece with others. For example, if there totally 100 pieces the shape of the matrix should be 100x100. In this case, the compute of the compatibility matrix will take some times when the puzzle is big.


In [ ]:
CM = pomeranz_CM(puzzle)

To place the piece, the puzzle solver just choose the first piece randomly and place it randomly, then compare all border which can near the piece placed with all others, then place the best one, which have best compatibility, to the slot. <br>And do it again and again until the board is full. <br>
We call this method as greedy_placer.

In [ ]:
puzzle.clean()
naiv_greedy_placer(puzzle, CM, display=False)
puzzle.display()

In [ ]:
# Improved version of the placer
puzzle.clean()
greedy_placer(puzzle, CM, rolling=False)
puzzle.display()

In [ ]:
# New version of the placer with the rolling behaviour
puzzle.clean()
greedy_placer(puzzle, CM, rolling=True)
puzzle.display()

## Segmenter

After the board is full, the puzzle solver will reuse the compatibility matrix to find the BestBuddies and just keep the best segmenter in board.

In [ ]:
BB = BestBuddies_matrix(CM)

In [ ]:
segment = segmenter(puzzle,BB)

In [ ]:
remove_all_but_segment(puzzle,segment)

In [ ]:
puzzle.display()